# Importação da informação

## Importar pacotes

In [1]:
#!pip install tmdbv3api

In [2]:
# Importar pacote de API do TMDB e baixar dados
import tmdbv3api as tmdb
tmdb_var = tmdb.TMDb()
tmdb_var.api_key = '1854c113335150e486ee88e252175599'

In [3]:
# Pandas
import pandas as pd

In [4]:
# Uso do sistema operacional
import os

In [5]:
import time

## Configurar TMBD e Parametrizar

In [6]:
# Configurações iniciais do pacote
tmdb_var.language = 'en'
tmdb_var.debug = True

In [7]:
# Variável movie
movie = tmdb.Movie()

## Fazer download dos dados conforme quantidade de IDs configurada
Por padrão, se extrai até o máximo ID criado

In [8]:
# Buscar o último filme lançado no site, e consequentemente, o último ID
max_id = movie.latest().id + 10
max_id
#max_id = 300 #Ajuste do max_id para fazer testes

1117248

In [17]:
movie_var = movie.details(615)
movie_var.popularity

104.873

In [9]:
# Gerar dataframes com os valores do cinema

def download_data_tmbd(tmdb_Movie_var, min_movie_id, max_movie_id, id_export = None, export_path = ''):

    ########################################################################################################################
    # CHECKS

    # Check if var is valid
    if str(type(tmdb_Movie_var)) != "<class 'tmdbv3api.objs.movie.Movie'>":
        print('tmdb_Movie_var value is not valid.')
        return

    # Limiting max_possible_movie_id
    max_possible_movie_id = movie.latest().id + 100
    if max_movie_id > max_possible_movie_id:
        print(f'max_movie_id adjusted from {max_movie_id} to {max_possible_movie_id}.')
        max_movie_id = max_possible_movie_id
    
    # Check if numbers are good
    if max_movie_id <= min_movie_id:
        print('max_movie_id must be greater than min_movie_id.')
        return

    # Check if numbers are good
    if id_export == None:
        id_export = ''
    else:
        id_export = '_' + str(id_export)

    # Check if var is valid
    if not export_path == '':
        export_path_plus = f"{export_path}\\"
    else:
        export_path_plus = ''
    
    
    ########################################################################################################################
    # Lista de atributos com valores singulares
    list_attr_movie = ['id','title','release_date','adult','budget','imdb_id','original_language','original_title','popularity','revenue','runtime','status','video','vote_average','vote_count','homepage']

    # Lista de atributos que retornam dicionários
    list_atrib_dict_movie = ['belongs_to_collection']

    # Lista de atributos que retornam lista de dicionários
    list_atrib_list_dict_movie = ['genres','production_companies','production_countries','spoken_languages']

    # Dataframe vazio com os valores singulares
    df_movies_list = pd.DataFrame(columns=list_attr_movie)

    # Função para ver se todos os valores de uma lista são dicionários
    def list_all_has_dict(list):
        return_var = False
        for value in list:
            if isinstance(dict(vars(tmdb_Movie_var)), dict):
                return_var = True
            else:
                return_var = False
                break
        return return_var

    # Dataframes a partir das variáveis de dicionário
    for valor in ['df_movie_' + attr_dict + ' = pd.DataFrame()' for attr_dict in list_atrib_dict_movie]:
        exec(valor)

    for valor in ['df_movie_' + attr_list_dict + ' = pd.DataFrame()' for attr_list_dict in list_atrib_list_dict_movie]:
        exec(valor)


    ########################################################################################################################
    # Download dos dados dos filmes para Dataframes a partir das variáveis de lista de dicionários
    max_progress = max_movie_id - min_movie_id + 1
    progress = 0
    progress_registers = 200
    print(f'Iniciando download dos dados dos filmes.')
    start_time = time.time()
    for i in range(min_movie_id, max_movie_id + 1):

        # Verificação de erros para prosseguir com o programa
        try:
            movie_var = tmdb_Movie_var.details(movie_id=i)

        except Exception:
            i

        else:

            # Valores valores no dataframe principal
            df_movies_list.loc[i] = [getattr(movie_var, attr) for attr in list_attr_movie]
            df_movies_list = df_movies_list.rename(columns={'id':'movie_id'}) #renomear a coluna 'id' para 'movie_id'

            # Salvar valores dos dataframes secundários - dicionários
            for attr_dict in list_atrib_dict_movie:
                
                value_attr = getattr(movie_var, attr_dict)
                if value_attr is not None:
                    value_attr = dict(vars(value_attr)) #Converter valores da lista para 'dict'

                    # Dataframe temporário com linha do filme
                    df_tmp_dict = pd.DataFrame([value_attr])

                    # Criando o movie_id e deixando-o na primeira coluna
                    df_tmp_dict['movie_id'] = i
                    first_column = df_tmp_dict.pop('movie_id')
                    df_tmp_dict.insert(0, 'movie_id', first_column)
                    del first_column

                    # Colocando a informação no dataset final
                    exec_value = 'df_movie_' + attr_dict + ' = pd.concat([df_movie_' + attr_dict + ', df_tmp_dict], ignore_index=True)'
                    exec(exec_value)
                    del df_tmp_dict

            # Salvar valores dos dataframes secundários - lista de dicionários
            for attr_list_dict in list_atrib_list_dict_movie:

                value_attr = getattr(movie_var, attr_list_dict)
                if type(value_attr) == list and list_all_has_dict(value_attr):
                    value_attr = [dict(vars(row)) for row in value_attr] #Converter valores da lista para 'dict'

                    # Dataframe temporário com linha do filme
                    df_tmp_dict = pd.DataFrame(value_attr)

                    # Criando o movie_id e deixando-o na primeira coluna
                    df_tmp_dict['movie_id'] = i
                    first_column = df_tmp_dict.pop('movie_id')
                    df_tmp_dict.insert(0, 'movie_id', first_column)
                    del first_column

                    # Colocando a informação no dataset final
                    exec_value = 'df_movie_' + attr_list_dict + ' = pd.concat([df_movie_' + attr_list_dict + ', df_tmp_dict], ignore_index=True)'
                    exec(exec_value)
                    del df_tmp_dict
                    
        # Check Status Steps
        if (i+1) >= (min_movie_id + (max_progress / progress_registers) * (progress + 1)):

            middle_time = time.time()
            elapsed_time = middle_time - start_time

            progress = progress + 1
            print(f'{(progress / progress_registers * 100):.1f}% do download feito. Último ID gerado: {i}; Filmes criados: {df_movies_list.shape[0]}.' + ' Tempo de processo: {:.2f} segundos'.format(elapsed_time))

            time.sleep(10.0)

        #print(f'Último ID gerado: {i}; Filmes criados: {df_movies_list.shape[0]}.')
        
        # Pause on proccess
        #time.sleep(0.5)

    df_movies_list = df_movies_list.dropna(subset=['title']).reset_index(drop=True)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'Download dos dados dos filmes concluído.' + ' Tempo de processo: {:.2f} segundos'.format(elapsed_time))


    #######################################################################################################################
    # Gerar lista dos datasets criados

    # df_movies_list
    list_datasets_tmbd = ['df_movies_list']

    # lista de bases de dicionários
    for attr_dict in list_atrib_dict_movie:
        list_datasets_tmbd.append(f'df_movie_{attr_dict}')

    # lista de bases de lista de dicionários
    for attr_list_dict in list_atrib_list_dict_movie:
        list_datasets_tmbd.append(f'df_movie_{attr_list_dict}')

    list_datasets_tmbd = pd.DataFrame(data=list_datasets_tmbd, columns=[['dataframe']])

    list_datasets_tmbd_export = list_datasets_tmbd['dataframe'] + f'{id_export}'

    print('\n\nLista de dataframes gerados.')
    display(list_datasets_tmbd)


    #######################################################################################################################
    # Fazer a print dos dados criados
    print('\nBase principal de filmes: df_movies_list')
    display(df_movies_list.head())

    for attr_dict in list_atrib_dict_movie:
        print(f'\nBase secundária: df_movie_{attr_dict}')
        exec(f'display(df_movie_{attr_dict}.head())')

    for attr_list_dict in list_atrib_list_dict_movie:
        print(f'\nBase secundária: df_movie_{attr_list_dict}')
        exec(f'display(df_movie_{attr_list_dict}.head())')


    #######################################################################################################################
    # Exportar bases

    # Criar export
    if export_path != '':
        if not os.path.exists(export_path):
            os.makedirs(export_path)

    # Exportar lista dos datasets criados
    list_datasets_tmbd.to_csv(f"{export_path_plus}list_datasets_tmbd{id_export}.csv", index=False, encoding="utf-8", header=False)

    # Exportar lista dos datasets criados
    df_movies_list.to_csv(f"{export_path_plus}df_movies_list{id_export}.csv", index=False, encoding="utf-8", header=True)

    for attr_dict in list_atrib_dict_movie:
        exec(f'df_movie_{attr_dict}.to_csv(f"{export_path_plus}df_movie_{attr_dict}{id_export}.csv", index=False, encoding="utf-8", header=True)')

    for attr_list_dict in list_atrib_list_dict_movie:
        exec(f'df_movie_{attr_list_dict}.to_csv(f"{export_path_plus}df_movie_{attr_list_dict}{id_export}.csv", index=False, encoding="utf-8", header=True)')

In [10]:
export_path = 'export_tmdb_database'
#download_data_tmbd(movie,1,100)

In [11]:
#download_data_tmbd(movie,1,100000,1,export_path)

In [22]:
#download_data_tmbd(movie,100001,200000,2,export_path)

Iniciando download dos dados dos filmes.
0.5% do download feito. Último ID gerado: 100500; Filmes criados: 317. Tempo de processo: 240.97 segundos
1.0% do download feito. Último ID gerado: 101000; Filmes criados: 538. Tempo de processo: 492.53 segundos
1.5% do download feito. Último ID gerado: 101500; Filmes criados: 772. Tempo de processo: 744.39 segundos
2.0% do download feito. Último ID gerado: 102000; Filmes criados: 1041. Tempo de processo: 995.69 segundos
2.5% do download feito. Último ID gerado: 102500; Filmes criados: 1303. Tempo de processo: 1249.14 segundos
3.0% do download feito. Último ID gerado: 103000; Filmes criados: 1563. Tempo de processo: 1502.21 segundos
3.5000000000000004% do download feito. Último ID gerado: 103500; Filmes criados: 1813. Tempo de processo: 1753.88 segundos
4.0% do download feito. Último ID gerado: 104000; Filmes criados: 2099. Tempo de processo: 2005.94 segundos
4.5% do download feito. Último ID gerado: 104500; Filmes criados: 2413. Tempo de proces

,dataframe
0,df_movies_list
1,df_movie_belongs_to_collection
2,df_movie_genres
3,df_movie_production_companies
4,df_movie_production_countries
5,df_movie_spoken_languages



Base principal de filmes: df_movies_list


,movie_id,title,release_date,adult,budget,imdb_id,original_language,original_title,popularity,revenue,runtime,status,video,vote_average,vote_count,homepage
0,100009,DevilDolls,2012-01-01,False,0,tt2492654,en,DevilDolls,1.554,0,90,Released,False,4.700,6,
1,100010,Flight Command,1940-12-27,False,0,tt0032477,en,Flight Command,1.196,0,116,Released,False,5.300,9,
2,100011,You Nazty Spy!,1940-01-19,False,0,tt0033281,en,You Nazty Spy!,1.216,0,18,Released,False,7.294,17,
3,100013,A Trip to Inside,2009-09-01,False,0,,en,A Trip to Inside,0.600,0,106,Released,False,0.000,0,
4,100016,Murder in the Air,1940-06-01,False,0,tt0032820,en,Murder in the Air,1.211,0,55,Released,False,4.500,4,



Base secundária: df_movie_belongs_to_collection


...

,movie_id,id,logo_path,name,origin_country
0,100009,15352,None,Full Moon Features,US
1,100010,21,/5Va1Ie5c4sjfEYqixQ3L8qg7fKu.png,Metro-Goldwyn-Mayer,US
2,100011,5,/lieeAioEBVsgsoDT9HICrdt5iRa.png,Columbia Pictures,US
3,100016,3245,/9dBTQp9XitrHkx20i1r9HhZ3Q6B.png,First National Pictures,US
4,100016,174,/IuAlhI9eVC9Z8UQWOIDdWRKSEJ.png,Warner Bros. Pictures,US



Base secundária: df_movie_production_countries


,movie_id,iso_3166_1,name
0,100010,US,United States of America
1,100011,US,United States of America
2,100013,DE,Germany
3,100016,US,United States of America
4,100017,DE,Germany



Base secundária: df_movie_spoken_languages


,movie_id,english_name,iso_639_1,name
0,100009,English,en,English
1,100010,English,en,English
2,100011,English,en,English
3,100013,German,de,Deutsch
4,100016,English,en,English


In [13]:
#download_data_tmbd(movie,200001,300000,3,export_path)

In [14]:
#download_data_tmbd(movie,300001,400000,4,export_path)

In [15]:
#download_data_tmbd(movie,400001,500000,5,export_path)

In [16]:
#download_data_tmbd(movie,500001,600000,6,export_path)

In [17]:
#download_data_tmbd(movie,600001,700000,7,export_path)

In [18]:
#download_data_tmbd(movie,700001,800000,8,export_path)

In [19]:
#download_data_tmbd(movie,800001,900000,9,export_path)

In [21]:
#download_data_tmbd(movie,900001,1000000,10,export_path)

In [1]:
#download_data_tmbd(movie,1000001,1100000,11,export_path)

In [24]:
#download_data_tmbd(movie,1100001,1200000,12,export_path)